#### Merging Files

In [ ]:
import pandas as pd 
file1 = pd.read_csv('aggregated_results_by_ds__pool_37687290__2023_02_11.tsv', sep='\t')
file2 = pd.read_csv('aggregated_results_by_ds__pool_37806285__2023_02_18.tsv', sep='\t')
file3 = pd.read_csv('aggregated_results_by_ds__pool_37939772__2023_02_27.tsv', sep='\t')

#### Testing the common sentences between files

In [ ]:
file1.shape

(91, 4)

In [ ]:
file2.shape

(600, 4)

In [ ]:
file3.shape

(600, 4)

In [ ]:
with open('aggregated_results_by_ds__pool_37687290__2023_02_11.tsv', 'r', encoding='utf-8') as f1, open('aggregated_results_by_ds__pool_37939772__2023_02_27.tsv', 'r', encoding='utf-8') as f2:
    sentences1 = set(line.strip().split('\t')[0] for line in f1)
    sentences2 = set(line.strip().split('\t')[0] for line in f2)

shared_sentences = sentences1.intersection(sentences2)

if shared_sentences:
    print("The following sentences are present in both files:")
    for sentence in shared_sentences:
        print(sentence)
else:
    print("There are no sentences in common between the two files.")


The following sentences are present in both files:
Китай тому пример , да и не только он .
Они раскрывались поутру в траве и под вечер увядали .
Мог иметься в виду и слесарь , и столяр .
Потому что хорошее должно быть и в виде приятного .
Сотрудница туалета закатила глаза и заговорила тихо и быстро .
Бухгалтеры звонят в редакцию и просят выяснить позицию фискалов .
Дело здесь не в оборудовании , а в другом .
Бесконечные стычки и распри , дележ и взаимные наговоры .
Госпиталь формировался в Курске от союза земств и городов .
Каждый человек рождается и умирает в своем энергетическом поле .
Пытался он внести и новые предложения в производство обуви .
Сейчас единства в этом вопросе в России не существует .
Секретарь вытаращил глаза на арестанта и не дописал слова .
Да и налоги в международных водах платить не надо .
Отец в первый же понедельник ушел добровольцем в ополчение .
Остальное - в основном всякая сборочная и ремонтная мелочь .
Да и обнаружилось это в тех авариях не сразу .
Национа

In [ ]:
print("the common sntences between 2023_02_11.tsv and 2023_02_18.tsv : ", len(shared_sentences))

the common sntences between 2023_02_11.tsv and 2023_02_18.tsv :  46


In [ ]:
print("the common sntences between 2023_02_11.tsv and 2023_02_27.tsv : ", len(shared_sentences))

the common sntences between 2023_02_11.tsv and 2023_02_27.tsv :  47


In [ ]:


annotated_data = pd.concat([file2, file3], axis = 0)
annotated_data = annotated_data.drop(annotated_data.columns[-1], axis=1)#annotated_data.iloc[:, :-1]
annotated_data.dropna(inplace = True, axis=1)
#annotated_data.to_csv('annotated_data.tsv', sep='\t', index=False, header = False)
annotated_data.columns = 'text cat_s confidence'.split()
annotated_data['category'] = annotated_data.cat_s.apply(lambda x: int(x[1]))
annotated_data = annotated_data.drop(annotated_data.columns[1], axis=1)

In [ ]:
annotated_data.shape

(1200, 3)

In [ ]:
annotated_data.head()

,text,confidence,category
0,"Разглядывая их под микроскопом , мы увидим при...",100.00%,4
1,"Иногда талантливые люди , не найдя себя в наук...",100.00%,4
2,Кандидаты в депутаты от общественных организац...,100.00%,6
3,"Главное , чтобы "" социальные лифты "" не остана...",100.00%,6
4,Франция признает российских студентов и не при...,100.00%,2


In [ ]:
#annotated_data.to_csv('full_annotated_data.tsv', index=False)

#Mapping 

In [ ]:
df = pd.read_csv('full_annotated_data.tsv')

df.head()

,text,confidence,category
0,"Разглядывая их под микроскопом , мы увидим при...",100.00%,4
1,"Иногда талантливые люди , не найдя себя в наук...",100.00%,4
2,Кандидаты в депутаты от общественных организац...,100.00%,6
3,"Главное , чтобы "" социальные лифты "" не остана...",100.00%,6
4,Франция признает российских студентов и не при...,100.00%,2


In [ ]:
import csv
sentence_to_columns = {}
sentences_tsv = []
OUTPUT_tsv = []
CONFIDENCE_tsv = []
with open('full_annotated_data.tsv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        sentences_tsv.append(row[0])
        OUTPUT_tsv.append(row[1])
        CONFIDENCE_tsv.append(row[2])
        sentence_to_columns[row[0]] = (row[1], row[2])

In [ ]:
sentences = sentences_tsv[1:]
OUTPUT = OUTPUT_tsv[1:]
CONFIDENCE = CONFIDENCE_tsv[1:]

In [ ]:
#len(sentences)
#len(OUTPUT)
len(CONFIDENCE)

1291

In [ ]:
%%capture

!pip install nltk conllu
import nltk
import conllu

from google.colab import drive
drive.mount('/content/drive')
data_train = "/content/drive/MyDrive/Ru_Syntagrus_Dataset/ru_syntagrus-ud-train.conllu"

In [ ]:
import pandas as pd 
# Read the UD Russian SynTagRus dataset in CONLU format
with open(data_train, "r") as file:
    data = conllu.parse(file.read())

In [ ]:
filtered_data = []
for sent in data:
    # Get the text of the sentence
    text = " ".join([token["form"] for token in sent])
    if text in sentences:
        filtered_data.append(sent)

In [ ]:
# Write the filtered sentences to a new CONLL-U file

with open('full_annotated_sentence.conllu', 'w') as f:
      f.writelines([sentence.serialize() + "\n" for sentence in filtered_data])

In [ ]:
## Test the new dataset 
with open("full_annotated_sentence.conllu", "r") as file:
    data = conllu.parse(file.read())

In [ ]:
data[0]

TokenList<За, неделю, начальник, управления, собирался, позвонить, Савельеву, ,, который, занимал, теперь, такой, же, пост, ,, как, и, Семен, Еремеевич, в, соседней, области, ,, и, узнать, о, деловых, качествах, Ефимовой, ., metadata={sent_id: "2003Anketa.xml_66", text: "За неделю начальник управления собирался позвонить Савельеву, который занимал теперь такой же пост, как и Семен Еремеевич в соседней области, и узнать о деловых качествах Ефимовой."}>

In [ ]:
len(data)

1200